In [1]:
from circuits import build_circuit
import numpy as np
import shadows as sh
import pandas as pd

def normalize_new_features(unnormalized_features):
    from sklearn.preprocessing import MinMaxScaler

    scaler = MinMaxScaler(feature_range=(0,1))

    normalized_features = scaler.fit_transform(unnormalized_features)
    return normalized_features
    

def run_pipeline(filename = None, ring_paulis = 'XY', entanglement = 'ring', num_layers = 1, encoding_axis = ("rx","ry"), train_test = 'train'):
    
    ring_paulis = ring_paulis.upper()
    df = pd.read_csv(f'../Data/{filename}')
    df = df.drop(df.columns[-1], axis = 1)


    data = df.to_numpy()
    n = len(data[0])


    circuits = [build_circuit(x, encoding_axes=encoding_axis, entanglement=entanglement, gate="cx", num_layers=num_layers)
                for x in data]

    paulis = sh.paulis_singles_xyz(n) + sh.paulis_ring_pairs(n, (ring_paulis[0], ring_paulis[1]))

    cfg = sh.ShadowConfig(T = 200, shots = 1000, seed = 123)

    new_features = sh.build_feature_matrix_from_circuits(circuits, paulis, cfg)
    
    normalized_features = normalize_new_features(new_features)
    
    df = pd.DataFrame(normalized_features)
    df.to_csv(f"../Data/{normalized_features.shape[1]}_features{ring_paulis[0]}{ring_paulis[1]}_{train_test}_QuantumLayers1.csv", index = False)


In [2]:
run_pipeline(filename = '../Data/X_train_scaled.csv', entanglement = 'full', train_test = 'train', num_layers = 1),
run_pipeline(filename = '../Data/X_test_scaled.csv', entanglement = 'full', train_test = 'test', num_layers = 1)
run_pipeline(filename = '../Data/X_val_scaled.csv', entanglement = 'full', train_test = 'val', num_layers = 1)

KeyboardInterrupt: 